Coding Discussions 4 <br>
Tianwei Liu

In [60]:
import pandas as pd
import numpy as np

In [183]:
## Read in the texts as strings
with open('aljazeera-khashoggi.txt', 'r') as file:
    text_alja = file.read().replace('\n', '')

with open('bbc-khashoggi.txt', 'r') as file:
    text_bbc = file.read().replace('\n', '')

with open('breitbart-khashoggi.txt', 'r') as file:
    text_breit = file.read().replace('\n', '')

with open('fox-khashoggi.txt', 'r') as file:
    text_fox = file.read().replace('\n', '')

with open('cnn-khashoggi.txt', 'r') as file:
    text_cnn = file.read().replace('\n', '')
    
stop_word = []
with open("stop_words.csv",mode='r') as file:
    data = csv.reader(file)
    for row in data:
        stop_word.extend(row)

In [54]:
import string

In [186]:
## We use a function to remove punctuation

def remove_punctuation(value):
    result = ""
    for c in value: 
        if c not in string.punctuation:
            result += c
    return result

In [187]:
## We now define a function that splits the text strings into words

def tokenize(text=None):
    text = text.lower()
    text = remove_punctuation(text) ## This step removes all punctuations from the text string
    text_list = text.split()
    text_list2 = [word for word in text_list if word not in stop_word]
    return text_list2

In [188]:
## Now we need to count the number of occurance of each word

def convert_text_to_dtm(txt):
    '''
    Converts text into a document term matrix.
    '''
    d = dict()
    for word in tokenize(txt):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)

In [189]:
# Build a funtion that counts the number of occurances of each word in texts
def gen_DTM(texts=None):
    '''
    Generate a document term matrix
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = convert_text_to_dtm(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True) # Fill in any missing values with 0s (i.e. when a word is in one text but not another)
    return DTM

D = gen_DTM([text_alja,text_bbc,text_breit,text_fox,text_cnn])

In [190]:
x0 = D.iloc[0].values
x1 = D.iloc[1].values
x2 = D.iloc[2].values
x3 = D.iloc[3].values
x4 = D.iloc[4].values
x_total= [x0,x1,x2,x3,x4]

In [191]:
## Define a function cosine to caltulate the cosine value
def cosine(a,b):
    cos = np.dot(a,b)/(np.sqrt(np.dot(a,a)) * np.sqrt(np.dot(b,b))  )
    return cos

In [192]:
## Obtain a cosine matrix (similarity)

result = []
for x in x_total:
    for y in x_total: 
        result.append(cosine(x,y).round(3))
result
result_dat = pd.Series(result)
result_dat.values.reshape(5,5)

array([[1.   , 0.679, 0.555, 0.658, 0.533],
       [0.679, 1.   , 0.565, 0.606, 0.504],
       [0.555, 0.565, 1.   , 0.518, 0.353],
       [0.658, 0.606, 0.518, 1.   , 0.514],
       [0.533, 0.504, 0.353, 0.514, 1.   ]])

Based on the matrix, we can see that no pair of two between these five texts are highly similar. The highest similary is between aljazeera and bbc. 

In [193]:
## Since these reports are all about Turkish President Erdogan addressing the 
## murder of journalist Jamal Khashoggi.
## I will also drop common words like Turkey, President, Erdogan, Journalist, Jamal, Khashoggi.

stop_word1 = []
with open("stop_words.csv",mode='r') as file:
    data = csv.reader(file)
    for row in data:
        stop_word1.extend(row)
stop_word1.extend(["turkey","president",'erdogan','journalist','jamal','khashoggi'])

In [194]:
## update function
def tokenize(text=None):
    text = text.lower()
    text = remove_punctuation(text) ## This step removes all punctuations from the text string
    text_list = text.split()
    text_list3 = [word for word in text_list if word not in stop_word1]
    return text_list3

In [195]:
## Recycle other process
def convert_text_to_dtm(txt):
    '''
    Converts text into a document term matrix.
    '''
    d = dict()
    for word in tokenize(txt):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)

def gen_DTM(texts=None):
    '''
    Generate a document term matrix
    '''
    DTM = pd.DataFrame()
    for text in texts:
        entry = convert_text_to_dtm(text)
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True) # Fill in any missing values with 0s (i.e. when a word is in one text but not another)
    return DTM

D = gen_DTM([text_alja,text_bbc,text_breit,text_fox,text_cnn])
x0 = D.iloc[0].values
x1 = D.iloc[1].values
x2 = D.iloc[2].values
x3 = D.iloc[3].values
x4 = D.iloc[4].values
x_total= [x0,x1,x2,x3,x4]

In [197]:
result1 = []
for x in x_total:
    for y in x_total: 
        result1.append(cosine(x,y).round(3))
result1
result1_dat = pd.Series(result1)
result1_dat.values.reshape(5,5)

array([[1.   , 0.651, 0.516, 0.572, 0.431],
       [0.651, 1.   , 0.513, 0.538, 0.407],
       [0.516, 0.513, 1.   , 0.471, 0.279],
       [0.572, 0.538, 0.471, 1.   , 0.35 ],
       [0.431, 0.407, 0.279, 0.35 , 1.   ]])

After removing soem common words, comparing with the matrix above, we can see that the similarity dropped significantly while the similarity between aljazeera and bbc is still the highest. The lowest similarity, in this case, happens between breitbart and cnn, which has a cos value of 0.279.